In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np
import glob
import nltk

In [5]:
files = glob.glob("musicas/charliebrownjr/*")
print('Músicas:')
print('------------------------------------------------------------------------------------------------------------------------------')
print(files)
print('------------------------------------------------------------------------------------------------------------------------------')

data = ''
for fname in files:
    with open(fname,'r') as f:
        data = data +' '+ f.read()

Músicas:
------------------------------------------------------------------------------------------------------------------------------
['musicas/charliebrownjr\\cbjr-ceu-azul.txt', 'musicas/charliebrownjr\\cbjr-como-tudo-deve-ser.txt', 'musicas/charliebrownjr\\cbjr-dias-luta-dias-gloria.txt', 'musicas/charliebrownjr\\cbjr-lugar-ao-sol.txt', 'musicas/charliebrownjr\\cbjr-meu-novo-mundo.txt', 'musicas/charliebrownjr\\cbjr-pontes-indestrutiveis.txt', 'musicas/charliebrownjr\\cbjr-proibida-pra-mim.txt', 'musicas/charliebrownjr\\cbjr-senhor-do-tempo.txt', 'musicas/charliebrownjr\\cbjr-so-os-loucos-sabem.txt', 'musicas/charliebrownjr\\cbjr-zoio-de-lula.txt']
------------------------------------------------------------------------------------------------------------------------------


In [8]:
tokenizer = Tokenizer()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1  #adding one for out-of-vocab words

# creating input sequences using list of tokens
input_sequences = []
for l in corpus:
    token_list = tokenizer.texts_to_sequences([l])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
input_sequences.append(n_gram_seq)


# implementing padding
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

# one hot code
label = ku.to_categorical(label, num_classes=total_words)

In [10]:
model= Sequential([
    Embedding(total_words, 100, input_length=max_seq_len-1), # -1 as the last word is the label
    Bidirectional(LSTM(150, return_sequences = True)),
    Dropout(0.2),  #drops 20% of units in a layer
    LSTM(100),
    Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 21, 100)           61100     
_________________________________________________________________
bidirectional (Bidirectional (None, 21, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 21, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 305)               30805     
_________________________________________________________________
dense_1 (Dense)              (None, 611)               186966    
Total params: 740,471
Trainable params: 740,471
Non-trainable params: 0
__________________________________________________

In [11]:
num_epochs= 100
history = model.fit(predictors, label, epochs=num_epochs, verbose=1)

Epoch 1/100
84/84 [==============================] - 7s 86ms/step - loss: 6.6096 - accuracy: 0.0350
Epoch 2/100
84/84 [==============================] - 7s 81ms/step - loss: 5.7117 - accuracy: 0.0376
Epoch 3/100
84/84 [==============================] - 7s 81ms/step - loss: 5.4351 - accuracy: 0.0342
Epoch 4/100
84/84 [==============================] - 7s 81ms/step - loss: 5.2900 - accuracy: 0.0414
Epoch 5/100
84/84 [==============================] - 7s 85ms/step - loss: 5.1519 - accuracy: 0.0500
Epoch 6/100
84/84 [==============================] - 7s 84ms/step - loss: 5.0392 - accuracy: 0.0504
Epoch 7/100
84/84 [==============================] - 7s 86ms/step - loss: 4.9691 - accuracy: 0.0515
Epoch 8/100
84/84 [==============================] - 7s 88ms/step - loss: 4.8744 - accuracy: 0.0635
Epoch 9/100
84/84 [==============================] - 7s 89ms/step - loss: 4.7849 - accuracy: 0.0654
Epoch 10/100
84/84 [==============================] - 8s 89ms/step - loss: 4.7131 - accuracy: 0.0673

84/84 [==============================] - 6s 75ms/step - loss: 1.4602 - accuracy: 0.7286
Epoch 82/100
84/84 [==============================] - 6s 76ms/step - loss: 1.4084 - accuracy: 0.7447
Epoch 83/100
84/84 [==============================] - 6s 74ms/step - loss: 1.3898 - accuracy: 0.7504
Epoch 84/100
84/84 [==============================] - 6s 75ms/step - loss: 1.4180 - accuracy: 0.7455
Epoch 85/100
84/84 [==============================] - 6s 75ms/step - loss: 1.3470 - accuracy: 0.7658
Epoch 86/100
84/84 [==============================] - 6s 75ms/step - loss: 1.3312 - accuracy: 0.7703
Epoch 87/100
84/84 [==============================] - 6s 75ms/step - loss: 1.3127 - accuracy: 0.7741
Epoch 88/100
84/84 [==============================] - 6s 75ms/step - loss: 1.3000 - accuracy: 0.7733
Epoch 89/100
84/84 [==============================] - 6s 75ms/step - loss: 1.2774 - accuracy: 0.7774
Epoch 90/100
84/84 [==============================] - 6s 74ms/step - loss: 1.2429 - accuracy: 0.7883
Epo

In [37]:
start_text = "eu te amo"
num_words = 100  #numero de palavras
  
for _ in range(num_words):
    token_list = tokenizer.texts_to_sequences([start_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    out_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            out_word = word
            break
    start_text += " " + out_word

In [36]:
print('Resultado:')
print('-------------------------------------------------------------------------------------------------------------------------------')
print(start_text)
print('-------------------------------------------------------------------------------------------------------------------------------')

Resultado:
-------------------------------------------------------------------------------------------------------------------------------
facil me deixa aqui a toa mal uso como voce mas em troca de uma foto com sorriso inocente nem pouco que olhando som por chover nossos nossos loucura um brown jr tudo que olhando sozinho fazer som som voce feliz guerra mais a vida tudo um brown brown brown brown brown fe a gente rala mais a gente cresce chover feito por de nossos nossos nossos brown brown brown brown brown brown boca tudo nossos sonhos nossos nossos brown brown brown brown brown brown brown brown brown brown brown brown brown brown brown brown brown brown brown brown brown brown
-------------------------------------------------------------------------------------------------------------------------------
